# End-to-end NLP: News Headline classifier

### Setup execution role and session

In [1]:
import numpy as np
import pandas as pd

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [2]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::322418637044:role/service-role/AmazonSageMaker-ExecutionRole-20190409T203908
CPU times: user 535 ms, sys: 66.3 ms, total: 601 ms
Wall time: 773 ms


### Download News Aggregator Dataset available at the public UCI dataset repository (these files should already be downloaded in previous notebook)

In [3]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

In [4]:
#!unzip NewsAggregatorDataset.zip

In [5]:
#!rm -rf __MACOSX/

In [6]:
#ls

### Let's visualize the dataset

We will load the newsCorpora.csv file to a Pandas dataframe for our data processing work

In [7]:
import pandas as pd
import mxnet
import re
import numpy as np
import os

In [9]:
column_names = ["TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
news_dataset = pd.read_csv('newsCorpora.csv', names=column_names, header=None, delimiter='\t')
news_dataset.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


#### For this exercice we'll only use the title (Headline) of the news story and the category as our target variable

In [10]:
df=news_dataset[['TITLE',"CATEGORY"]]

In [11]:
from collections import Counter
Counter(df['CATEGORY'])

Counter({'b': 115967, 't': 108344, 'e': 152469, 'm': 45639})

The dataset has four categories: Business (b), Science & Technology (t), Entertainment (e) and Health & Medicine (m).

## Natural Language pre processing

We will do some basic processing of the text data to convert it into numerical form that the algorithm will be able to consume to create a model.
We will do typical pre processing for NLP workloads such as: dummy encoding the labels, tokenizing the documents and set fixed sequence lengths for input feature dimension, padding documents to have fixed length input vectors.

#### Dummy encode the labels

In [12]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
encoder = preprocessing.LabelEncoder()

docs = df["TITLE"].values

encoder.fit(df["CATEGORY"].values)
encoded_Y = encoder.transform(df["CATEGORY"].values)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

Using MXNet backend


In [13]:
#bucket = <bucket> # custom bucket name.
s3_bucket = sess.default_bucket()
s3_prefix = 'news'

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-322418637044


In [14]:
list(encoder.classes_)

['b', 'e', 'm', 't']

In [15]:
encoded_Y

array([0, 0, 0, ..., 2, 2, 2])

#### Tokenize documents and set fixed sequence lengths for input feature dimension.

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print("Vocabulary size: " + str(vocab_size))
# pad documents to a max length of 4 words
max_length = 40
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Number of headlines: " + str(len(padded_docs)))

Vocabulary size: 75286
Number of headlines: 422419


In [17]:
docs[0]

'Fed official says weak data caused by weather, should not slow taper'

### Import word embeddings

In [18]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./vectors.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 71291 word vectors.


##### Create embedding matrix

In [19]:
#embeddings_index

In [20]:
#print(t.word_index)

In [21]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [22]:
mkdir ./data/ ./data/embeddings/

### Save embedding matrix to push to S3 for Sagemaker to use during training

In [24]:
#embedding_matrix.dump("ingredients-embedding-matrix.dat")
np.save(file="./data/embeddings/docs-embedding-matrix",
        arr=embedding_matrix,
        allow_pickle=False)
print(embedding_matrix.shape)

(75286, 100)


### Train, test split

In this section we will prep the data for ingestion for the algortihm. Split the data set in train and test samples and uplad the data to S3

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_docs, dummy_y, test_size=0.2, random_state=42)

In [26]:
!mkdir data/train/ data/test/

In [27]:
np.save('./data/train/train_X.npy', X_train)
np.save('./data/train/train_Y.npy', y_train)
np.save('./data/test/test_X.npy', X_test)
np.save('./data/test/test_Y.npy', y_test)

In [28]:
traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)
embeddings_s3_prefix='{}/data/embeddings'.format(s3_prefix)
output_s3 = 's3://{}/{}/models/'.format(s3_bucket, s3_prefix)
code_location_s3 = 's3://{}/{}/codes'.format(s3_bucket, s3_prefix)

In [29]:
train_s3 = sess.upload_data(path='./data/train/', bucket=s3_bucket, key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', bucket=s3_bucket, key_prefix=testdata_s3_prefix)
embeddings_s3 = sess.upload_data(path='./data/embeddings/', bucket=s3_bucket, key_prefix=embeddings_s3_prefix)


In [30]:
inputs = {'train':train_s3, 'test': test_s3, 'embeddings': embeddings_s3}

print(inputs)

{'train': 's3://sagemaker-us-east-1-322418637044/news/data/train', 'test': 's3://sagemaker-us-east-1-322418637044/news/data/test', 'embeddings': 's3://sagemaker-us-east-1-322418637044/news/data/embeddings'}


## Sagemaker training with differentiated infrastructure

We will use the high level MXNet SDK to train our MXNet model using Sagemaker. We have packaged the code we used to build and train our model in the previous notebook (headline-classifier-local.ipynb). The training script is available in the tf-src directory. You can read the details of the script and format in the python file: keras_script_mxnet.py.

In [31]:
import sagemaker
from sagemaker.mxnet import MXNet

### Define hyperparameters to push to algorithm

In [32]:
hyperparameters = {'epochs': 5, 'vocab_size':vocab_size, 'num_classes':encoder.classes_.size}

We have our `MXNet` estimator object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only  remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `MXNet` estimator classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

We will run the training job using a ml.p2.xlarge instance (GPUs) to accelerate our training. If your account runs into resource limits please use a ml.c5.xlarge instance. 

In [ ]:
mxnet_estimator = MXNet(entry_point='keras_script_mxnet.py',
                       source_dir='./tf-src',
                        role=role,
                        train_instance_type='ml.p2.xlarge',
                        train_instance_count=1,
                        framework_version='1.3.0',
                        py_version='py3',
                        hyperparameters=hyperparameters)
mxnet_estimator.fit(inputs)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-322418637044
INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2019-04-09-14-32-47-124


2019-04-09 14:32:47 Starting - Starting the training job...
2019-04-09 14:32:51 Starting - Launching requested ML instances......
2019-04-09 14:33:58 Starting - Preparing the instances for training.....
2019-04-09 14:34:57,956 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2019-04-09 14:34:57,959 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-04-09 14:34:57,973 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_CHANNEL_TRAIN': '/opt/ml/input/data/train', 'SM_FRAMEWORK_MODULE': 'sagemaker_mxnet_container.training:main', 'SM_USER_ARGS': '["--epochs","5","--num_classes","4","--vocab_size","75286"]', 'SM_HP_EPOCHS': '5', 'SM_USER_ENTRY_POINT': 'keras_script_mxnet.py', 'SM_MODEL_DIR': '/opt/ml/model', 'SM_HPS': '{"epochs":5,"num_classes":4,"vocab_size":75286}', 'SM_TRAINING_ENV': '{"additional_framework_parameters":{},"channel_input_dirs":{"embeddings":"/opt/ml/input/data/embeddings","test":"/o


2019-04-09 14:34:58 Downloading - Downloading input data
2019-04-09 14:34:58 Training - Training image download completed. Training in progress. - 298s - loss: 0.3241 - acc: 0.8608
Epoch 2/5
 - 301s - loss: 0.3052 - acc: 0.8696
Epoch 3/5


## Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same type of instance that we used to train. Because instance endpoints will be up and running for long, it's advisable to choose a cheaper instance for inference.

We will deploy our model using the MXNetModel object that will recieve as input the model object output by the training job to S3. We can use the different features of Sagemaker as decoupled modules. You can bring a model trained outside of Sagemaker and deploy it using the Sagemaker model deployment api.

In [29]:
import boto3
s3 = boto3.resource('s3')

key = mxnet_estimator.model_data[mxnet_estimator.model_data.find("/", 5)+1:]
s3.Bucket(s3_bucket).download_file(key, 'model.tar.gz')

In [30]:
model_path='model.tar.gz'
from sagemaker.mxnet import MXNet, MXNetModel

sagemaker_model = MXNetModel(model_data = model_path,
                             role = role,
                             entry_point = 'default_classifier.py',
                             py_version='py3')

In [31]:
predictor = mxnet_estimator.deploy(initial_instance_count=1,
                           instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2019-04-02-16-54-36-116
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2019-04-02-16-54-36-116


---------------------------------------------------------------------------!

### Your model should now be in production as a RESTful API!

In [32]:
display(predictor.accept, predictor.content_type, predictor.deserializer, predictor.endpoint, predictor.sagemaker_session, predictor.serializer)

'application/json'

'application/json'

'sagemaker-mxnet-2019-04-02-16-54-36-116'

Let's evaluate our model with an example headline. You can be creative with your headlines.

In [ ]:
example_doc=['Senate prepares to vote on dueling plans to end shutdown']
# integer encode the document
encoded_example = t.texts_to_sequences(example_doc)

# pad documents to a max length of 4 words
max_length = 40
padded_example = pad_sequences(encoded_example, maxlen=max_length, padding='post')

The result represents the probability or confidence of the model that the headline represents one of the classes: 'b', 'e', 'm', 't'

In [35]:
predictor.predict(padded_example.tolist())

[[0.9064793586730957,
  0.005665271542966366,
  0.006633899174630642,
  0.08122153580188751]]

## HPO

In [41]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [42]:
#hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.01, 0.2)}
hyperparameter_ranges = {'epochs': IntegerParameter(5, 10)}

In [43]:
objective_metric_name = 'loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss = ([0-9\\.]+)'}]

In [47]:
hyperparameters = {'epochs': 5, 'vocab_size':vocab_size, 'num_classes':encoder.classes_.size}

In [48]:
mxnet_estimator = MXNet(entry_point='keras_script_mxnet.py',
                       source_dir='./tf-src',
                        role=role,
                        train_instance_type='ml.m4.xlarge',
                        train_instance_count=1,
                        framework_version='1.3.0',
                        py_version='py3',
                        hyperparameters=hyperparameters)

In [51]:
tuner = HyperparameterTuner(mxnet_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=5,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

In [52]:
tuner.fit(inputs)

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-mxnet-190402-1720
